# Day 6: Universal Orbit Map

https://adventofcode.com/2019/day/6

## Setup

In [1]:
TEXT = readlines("input/06.txt");

In [2]:
typeof(TEXT)

Array{String,1}

## Implementation

In [3]:
mutable struct Node
    name::String
    parent
    children::Array
end
Node(name) = Node(name, nothing, [])
ancestors(node::Node) = vcat([node], ancestors(node.parent))
ancestors(node::Nothing) = []
nameof(node::Node) = node.name

function prettyprint(node::Node, depth=0)
    println(repeat("| ", depth) * " + $(node.name)")
    for child in node.children
        prettyprint(child, depth+1)
    end
end
;

In [4]:
struct OrbitNet
    all::Dict{String,Node}
end

function buildnet(text)
    orbits = [split(line, ")") for line in text]
    all_keys = Set(vcat(orbits...))
    all = Dict(k => Node(k) for k in all_keys)
    for (p, c) in orbits
        parent, child = all[p], all[c]
        push!(parent.children, child)
        child.parent = parent
    end
    OrbitNet(all)
end
prettyprint(on::OrbitNet) = prettyprint(on["COM"])

Base.getindex(on::OrbitNet, key) = on.all[key]
;

In [5]:
countall(node, depth=0) = depth + sum(Int[countall(child, depth+1) for child in node.children])
countall(on::OrbitNet) = countall(on["COM"])
;

In [6]:
length([1,2,3])

3

In [7]:
function shortestpath(node1::Node, node2::Node)
    # find root
    anc1, anc2 = reverse(ancestors(node1)), reverse(ancestors(node2))
    @assert anc1[1] === anc2[1]
    i = 1
    while anc1[i] == anc2[i] && i < length(anc1) && i < length(anc2)
        i += 1
    end
    length(anc1) + length(anc2) - 2*(i-1)
end
;

## Tests

In [8]:
test = split("""COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L""", "\n");

In [9]:
on = buildnet(test);

In [10]:
prettyprint(on)

 + COM
|  + B
| |  + C
| | |  + D
| | | |  + E
| | | | |  + F
| | | | |  + J
| | | | | |  + K
| | | | | | |  + L
| | | |  + I
| |  + G
| | |  + H


In [11]:
countall(on)

42

In [12]:
shortestpath(on["L"], on["H"])

8

## Prep

In [13]:
ON = buildnet(TEXT);

## Part One

In [14]:
countall(ON)

358244

In [15]:
# don't count self->parent as "orbital transfer"
shortestpath(ON["YOU"], ON["SAN"]) - 2

517